In [1]:
import os
import json
from glob import glob
from pathlib import Path
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
output_path = './bbox_results'

In [3]:
# bbox 그리기 및 저장 함수 정의
def draw_and_save_bbox(nation_code, input_base_path='./code/data', output_base_path='./bbox_output'):
    """
    지정된 국가 코드의 데이터셋에 대해 bbox를 그리고 저장하는 함수
    
    Args:
        nation_code (str): 국가 코드 ('vi', 'th', 'zh', 'ja' 중 하나)
        input_base_path (str): 입력 데이터가 있는 기본 경로
        output_base_path (str): 출력 이미지를 저장할 기본 경로
    """
    # 국가별 폴더명 매핑
    nation_dict = {
        'vi': 'vietnamese_receipt',
        'th': 'thai_receipt',
        'zh': 'chinese_receipt',
        'ja': 'japanese_receipt',
    }
    
    if nation_code not in nation_dict:
        raise ValueError(f"Invalid nation code. Must be one of {list(nation_dict.keys())}")
    
    # 경로 설정
    nation_folder = nation_dict[nation_code]
    json_path = os.path.join(input_base_path, nation_folder, 'ufo/train.json')
    img_input_path = os.path.join(input_base_path, nation_folder, 'img/train')
    img_output_path = os.path.join(output_base_path, nation_folder)
    
    # 출력 디렉토리 생성
    os.makedirs(img_output_path, exist_ok=True)
    
    # JSON 데이터 로드
    with open(json_path) as f:
        json_data = json.load(f)['images']
    
    # 각 이미지에 대해 처리
    for img_name in json_data.keys():
        # 원본 이미지 로드
        img = Image.open(os.path.join(img_input_path, img_name))
        
        # matplotlib figure 생성
        fig, ax = plt.subplots(figsize=(12, 7))
        ax.imshow(img)
        
        # 모든 단어에 대해 bbox 그리기
        for word_info in json_data[img_name]['words'].values():
            points = word_info['points']
            x1, y1 = points[0]
            x2, y2 = points[2]
            rect = patches.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                linewidth=1,
                edgecolor='r',
                facecolor='none'
            )
            ax.add_patch(rect)
        
        # 축 제거
        ax.set_xticks([])
        ax.set_yticks([])
        
        # 여백 최소화
        plt.tight_layout()
        
        # 이미지 저장
        output_filename = os.path.splitext(img_name)[0] + '_bbox.png'
        plt.savefig(
            os.path.join(img_output_path, output_filename),
            bbox_inches='tight',
            pad_inches=0,
            dpi=100
        )
        
        # 메모리 정리
        plt.close(fig)
        
    print(f"Completed processing {len(json_data)} images for {nation_folder}")
    print(f"Output saved to: {img_output_path}")

In [4]:
# 단일 국가 처리 예시
nation_code = 'zh'  # 원하는 국가 코드로 변경 가능 ('vi', 'th', 'zh', 'ja')
draw_and_save_bbox(nation_code, output_base_path=output_path)

Completed processing 100 images for chinese_receipt
Output saved to: ./bbox_results/chinese_receipt


In [5]:
"""
# 모든 국가 처리 예시
output_path = './bbox_results'
for nation_code in ['vi', 'th', 'zh', 'ja']:
    try:
        draw_and_save_bbox(nation_code, output_base_path=output_path)
    except Exception as e:
        print(f"Error processing {nation_code}: {str(e)}")
"""

'\n# 모든 국가 처리 예시\noutput_path = \'./bbox_results\'\nfor nation_code in [\'vi\', \'th\', \'zh\', \'ja\']:\n    try:\n        draw_and_save_bbox(nation_code, output_base_path=output_path)\n    except Exception as e:\n        print(f"Error processing {nation_code}: {str(e)}")\n'